In [4]:
from vosk import Model, KaldiRecognizer

In [5]:
#Importing the files and setting up the requirements
FRAME_RATE=16000
CHANNELS=1

model=Model(model_name="vosk-model-en-us-0.22")
rec=KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

In [6]:
from pydub import AudioSegment

In [6]:
#Importing the audio file
mp3=AudioSegment.from_mp3('marketplace.mp3')
mp3=mp3.set_channels(CHANNELS)
mp3=mp3.set_frame_rate(FRAME_RATE)

In [7]:
#Converting the audio to words
rec.AcceptWaveform(mp3.raw_data)
result=rec.Result()
result

'{\n  "result" : [{\n      "conf" : 1.000000,\n      "end" : 0.180000,\n      "start" : 0.090000,\n      "word" : "the"\n    }, {\n      "conf" : 1.000000,\n      "end" : 0.570000,\n      "start" : 0.180000,\n      "word" : "funny"\n    }, {\n      "conf" : 1.000000,\n      "end" : 0.960000,\n      "start" : 0.600000,\n      "word" : "thing"\n    }, {\n      "conf" : 1.000000,\n      "end" : 1.230000,\n      "start" : 0.960000,\n      "word" : "about"\n    }, {\n      "conf" : 1.000000,\n      "end" : 1.320000,\n      "start" : 1.230000,\n      "word" : "the"\n    }, {\n      "conf" : 1.000000,\n      "end" : 1.680000,\n      "start" : 1.320000,\n      "word" : "big"\n    }, {\n      "conf" : 1.000000,\n      "end" : 2.220000,\n      "start" : 1.710000,\n      "word" : "economic"\n    }, {\n      "conf" : 1.000000,\n      "end" : 2.490000,\n      "start" : 2.220000,\n      "word" : "news"\n    }, {\n      "conf" : 1.000000,\n      "end" : 2.580000,\n      "start" : 2.490000,\n      "wo

In [8]:
#Converting json file to normal readable text
import json
text=json.loads(result)["text"]
text

"the funny thing about the big economic news of the day the fed raising interest rates half a percentage point was that there was only really one tidbit of actual news in the news and the interest rate increase wasn't it you knew it was coming i knew it was common wall street news come and businesses knew it was common so on this fed day on this program something a little bit different jay powell in his own words five of 'em his most used economic words from today's press conference were number one of course it's the biggie two percent inflation inflation inflation inflation inflation inflation lh dealing with inflation pals big worry the thing keeping him up at night price stability is the fed's whole ballgame right now pau basically said as much today we're"

In [7]:
import subprocess
cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=text)

In [29]:
#Loading the pegasus model

from transformers import PegasusForConditionalGeneration, PegasusTokenizer,AutoTokenizer



In [30]:
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

In [22]:
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

In [32]:
# Create tokens - number representation of our text
tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")
# Input tokens
tokens

{'input_ids': tensor([[  109,  3765,   474,   160,   109,   461,  1500,   990,   113,   109,
           242,   109,  7076,  4673,   820,  1372,   751,   114,  3622,   491,
           140,   120,   186,   140,   209,   288,   156, 75904,   113,  1916,
           990,   115,   109,   990,   111,   109,   820,   872,   815,  1417,
           131,   144,   126,   119,  1606,   126,   140,   792,   532,  1606,
           126,   140,   830,  1075,  1913,   990,   331,   111,  1098,  1606,
           126,   140,   830,   167,   124,   136,  7076,   242,   124,   136,
           431,   364,   114,   332,   588,   291, 55144, 14750,  3422,   115,
           169,   282,   989,   668,   113,  1034,  4192,   169,   205,   263,
          1500,   989,   135,   380,   131,   116,  1833,  1792,   195,   344,
           156,   113,   422,   126,   131,   116,   109, 71249,   228,   837,
          7615,  7615,  7615,  7615,  7615,  7615,   110, 60693,  3297,   122,
          7615, 27140,   461,  2262,  

In [33]:
# Summarize 
summary = model.generate(**tokens)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [34]:
# Output summary tokens
summary[0]
# Decode summary
tokenizer.decode(summary[0])

"<pad> On this fed day on this program jay powell in his own words five of 'em his most used economic words from today's press conference.</s>"

In [8]:
#Creating a single function to do all task
def voice_recognition(filename):
    model = Model(model_name="vosk-model-en-us-0.22")
    rec = KaldiRecognizer(model, FRAME_RATE)
    rec.SetWords(True)
    
    mp3 = AudioSegment.from_mp3(filename)
    mp3 = mp3.set_channels(CHANNELS)
    mp3 = mp3.set_frame_rate(FRAME_RATE)
    
    step = 45000
    transcript = ""
    for i in range(0, len(mp3), step):
        print(f"Progress: {i/len(mp3)}")
        segment = mp3[i:i+step]
        rec.AcceptWaveform(segment.raw_data)
        result = rec.Result()
        text = json.loads(result)["text"]
        transcript += text
    
    cased = subprocess.check_output('python recasepunc/recasepunc.py predict recasepunc/checkpoint', shell=True, text=True, input=transcript)
    return cased

In [ ]:
#Applying the model
transcript = voice_recognition("marketplace_full.mp3")
transcript

In [1]:
#Summarizaring the text
from transformers import pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
split_tokens = transcript.split(" ")
docs = []
for i in range(0, len(split_tokens), 850):
    selection = " ".join(split_tokens[i:(i+850)])
    docs.append(selection)



In [ ]:
#Printing the summary
summaries = summarizer(docs)
summary = "\n\n".join([d["summary_text"] for d in summaries])
print(summary)